<a href="https://colab.research.google.com/github/chrishg23-jpg/HES-benchmark/blob/main/HES2full8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# HES_26.0_B_PERTURBATION_TEST.py: Act XXVI-B - Sensitivity Analysis

import numpy as np
import time
import warnings
warnings.filterwarnings("ignore", category=RuntimeWarning)
np.random.seed(42)

# --- 1. PERTURBED PARAMETERS ---
GRID_SIZE = 100
ALPHA = 3.00
# Delta is perturbed by ~+1%
DELTA_PERTURBED = 4.99
BETA, GAMMA = 0.1, 0.001
L_SQUARE = 1.0
STEPS = 500 # Extended Run for phase space mapping
SUBSYSTEM_SIZE = 3
FIT_START, FIT_END = 20, 50 # For lambda calculation

# --- CORE SIMULATION FUNCTIONS (Same as HES 26.0) ---
# NOTE: The full von_neumann_entropy, laplacian_2d_numpy, etc., are included here
# to ensure the physics simulation runs correctly.

def von_neumann_entropy(subsystem):
    # (function code for von_neumann_entropy...)
    flat = subsystem.flatten()
    rho = np.outer(flat, flat)
    rho /= np.trace(rho + 1e-10)
    eigenvalues = np.linalg.eigvalsh(rho)
    eigenvalues = eigenvalues[eigenvalues > 1e-10]
    return -np.sum(eigenvalues * np.log2(eigenvalues)) if len(eigenvalues) > 0 else 0.0

def laplacian_2d_numpy(field):
    # (function code for laplacian_2d_numpy...)
    S_padded = np.pad(field, 1, mode='wrap')
    S_center = S_padded[1:-1, 1:-1]
    L = S_padded[2:, 1:-1] + S_padded[:-2, 1:-1] + S_padded[1:-1, :-2] + S_padded[1:-1, 2:] - 4 * S_center
    return L

# --- 2. RUN SIMULATION & METRIC CALCULATION ---
def run_simulation_perturbed(alpha, delta, grid_size, total_steps):
    s = np.random.uniform(-1, 1, (grid_size, grid_size))
    entropy_map = np.zeros((grid_size, grid_size))
    C_history = []

    for step in range(total_steps):
        # 1. Compute Entropy Map
        for i in range(grid_size):
            for j in range(grid_size):
                i_start, i_end = max(0, i - SUBSYSTEM_SIZE//2), min(grid_size, i + SUBSYSTEM_SIZE//2 + 1)
                j_start, j_end = max(0, j - SUBSYSTEM_SIZE//2), min(grid_size, j + SUBSYSTEM_SIZE//2 + 1)
                if i_end > i_start and j_end > j_start:
                    entropy_map[i,j] = von_neumann_entropy(s[i_start:i_end, j_start:j_end])

        # 2. Compute Curvature
        curvature = laplacian_2d_numpy(entropy_map)
        curvature_norm = curvature / L_SQUARE

        # 3. Update Rule with PERTURBED DELTA
        nabla_s_approx = (s - (np.roll(s, 1, axis=0) + np.roll(s, -1, axis=0) +
                               np.roll(s, 1, axis=1) + np.roll(s, -1, axis=1)) / 4.0)
        s_new = s - (alpha * nabla_s_approx + BETA * s + GAMMA * np.mean(s) * s + delta * curvature_norm)
        s = np.clip(s_new, -1, 1)
        C_history.append(np.mean(np.abs(curvature_norm)))

    # --- POST-RUN METRICS ---
    # G_eff Calculation
    boundary_area = 4 * grid_size
    boundary_entropy = np.mean([entropy_map[0,:], entropy_map[-1,:],
                                entropy_map[:,0], entropy_map[:,-1]])
    G_eff = boundary_area / (4 * boundary_entropy) if boundary_entropy > 0 else 1.0e+20

    # Lambda_sim Calculation
    lambda_sim = 0.0
    C_data = np.array(C_history[FIT_START:FIT_END])
    if C_data.size >= 2 and np.max(C_data) > 1e-18:
        log_C = np.log(C_data + 1e-18)
        time_steps = np.arange(len(log_C))
        poly_coeffs = np.polyfit(time_steps, log_C, 1)
        lambda_sim = -poly_coeffs[0]

    return G_eff, lambda_sim


# --- 3. EXECUTION ---
print(f"Starting HES 26.0-B Perturbation Test (N=100, δ={DELTA_PERTURBED:.2f}).")
start_time = time.time()
G_eff_result, lambda_sim_result = run_simulation_perturbed(ALPHA, DELTA_PERTURBED, GRID_SIZE, STEPS)
end_time = time.time()
elapsed_time = end_time - start_time

# --- 4. FINAL REPORT OUTPUT ---
print(f"Simulation Finished in {elapsed_time:.2f} seconds.")
print("\n" + "="*80)
print("ACT XXVI-B: PERTURBATION TEST REPORT (Hyper-Sensitivity Check)")
print(f"PERTURBED PARAMS: α={ALPHA:.2f}, δ={DELTA_PERTURBED:.2f}, ρ={DELTA_PERTURBED/ALPHA:.3f}")
print("==================================================================================")
print(f"1. Final Estimated G: {G_eff_result:.2e}")
print(f"2. Curvature Decay (λ_sim): {lambda_sim_result:.6f}")
print("-" * 80)
print("ASSESSMENT: Analyzing the effect of the +1.01% Contraction Force increase...")


Starting HES 26.0-B Perturbation Test (N=100, δ=4.99).
Simulation Finished in 235.77 seconds.

ACT XXVI-B: PERTURBATION TEST REPORT (Hyper-Sensitivity Check)
PERTURBED PARAMS: α=3.00, δ=4.99, ρ=1.663
1. Final Estimated G: 6.93e+11
2. Curvature Decay (λ_sim): 0.276665
--------------------------------------------------------------------------------
ASSESSMENT: Analyzing the effect of the +1.01% Contraction Force increase...
